In [40]:
import pandas as pd
import numpy as np
import random

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split

from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB

from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import classification_report
from sklearn.model_selection import RandomizedSearchCV

import warnings
warnings.filterwarnings('ignore')

df = pd.read_csv("https://raw.githubusercontent.com/4GeeksAcademy/naive-bayes-project-tutorial/main/playstore_reviews.csv")
df.head()

,package_name,review,polarity
0,com.facebook.katana,privacy at least put some option appear offli...,0
1,com.facebook.katana,"messenger issues ever since the last update, ...",0
2,com.facebook.katana,profile any time my wife or anybody has more ...,0
3,com.facebook.katana,the new features suck for those of us who don...,0
4,com.facebook.katana,forced reload on uploading pic on replying co...,0


In [27]:
df["review"] = df["review"].str.strip().str.lower()
df.head()

,package_name,review,polarity
0,com.facebook.katana,privacy at least put some option appear offlin...,0
1,com.facebook.katana,"messenger issues ever since the last update, i...",0
2,com.facebook.katana,profile any time my wife or anybody has more t...,0
3,com.facebook.katana,the new features suck for those of us who don'...,0
4,com.facebook.katana,forced reload on uploading pic on replying com...,0


In [28]:
X = df["review"]
y = df["polarity"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

X_train.head()

331    just did the latest update on viber and yet ag...
733    keeps crashing it only works well in extreme d...
382    the fail boat has arrived the 6.0 version is t...
704    superfast, just as i remember it ! opera mini ...
813    installed and immediately deleted this crap i ...
Name: review, dtype: object

In [ ]:

vectorizer = TfidfVectorizer()

X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

In [30]:
X_train_tfidf

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 23060 stored elements and shape (712, 3553)>

In [31]:
model = MultinomialNB()

model.fit(X_train_tfidf, y_train)


MultinomialNB()

In [32]:
y_pred_train = model.predict(X_train_tfidf)
y_pred_test = model.predict(X_test_tfidf)

In [33]:
accuracy_train = accuracy_score(y_train, y_pred_train)
accuracy_test = accuracy_score(y_test, y_pred_test)

print("Accuracy Test: ", accuracy_test)
print("Accuracy Train: ", accuracy_train)

Accuracy Test:  0.7653631284916201
Accuracy Train:  0.8047752808988764


In [34]:
model = GaussianNB()

model.fit(X_train_tfidf.toarray(), y_train)

GaussianNB()

In [35]:
y_pred_train = model.predict(X_train_tfidf.toarray())
y_pred_test = model.predict(X_test_tfidf.toarray())

In [36]:
accuracy_train = accuracy_score(y_train, y_pred_train)
accuracy_test = accuracy_score(y_test, y_pred_test)

print("Accuracy Train:", accuracy_train)
print("Accuracy Test:", accuracy_test)

Accuracy Train: 0.9915730337078652
Accuracy Test: 0.8156424581005587


In [37]:
model = BernoulliNB()

model.fit(X_train_tfidf.toarray(), y_train)

BernoulliNB()

In [38]:
y_pred_train = model.predict(X_train_tfidf.toarray())
y_pred_test = model.predict(X_test_tfidf.toarray())

In [39]:
accuracy_train = accuracy_score(y_train, y_pred_train)
accuracy_test = accuracy_score(y_test, y_pred_test)

print("Accuracy Train:", accuracy_train)
print("Accuracy Test:", accuracy_test)

Accuracy Train: 0.9592696629213483
Accuracy Test: 0.8491620111731844


El mejor modelo seria el de Bernoulli

In [ ]:
#OPTIMIZACION

hyperparams = {
    "alpha": np.linspace(0.01, 10.0, 200),
    "binarize": [True, False]
}

random_search = RandomizedSearchCV(model, hyperparams, n_iter = 50, scoring = "accuracy", cv = 5, random_state = 42)
random_search

RandomizedSearchCV(cv=5, estimator=BernoulliNB(), n_iter=50,
                   param_distributions={'alpha': array([ 0.01      ,  0.06020101,  0.11040201,  0.16060302,  0.21080402,
        0.26100503,  0.31120603,  0.36140704,  0.41160804,  0.46180905,
        0.51201005,  0.56221106,  0.61241206,  0.66261307,  0.71281407,
        0.76301508,  0.81321608,  0.86341709,  0.91361809,  0.9638191 ,
        1.0140201 ,  1.06422111,  1.11442211,  1.16462312,...
        8.54417085,  8.59437186,  8.64457286,  8.69477387,  8.74497487,
        8.79517588,  8.84537688,  8.89557789,  8.94577889,  8.9959799 ,
        9.0461809 ,  9.09638191,  9.14658291,  9.19678392,  9.24698492,
        9.29718593,  9.34738693,  9.39758794,  9.44778894,  9.49798995,
        9.54819095,  9.59839196,  9.64859296,  9.69879397,  9.74899497,
        9.79919598,  9.84939698,  9.89959799,  9.94979899, 10.        ]),
                                        'binarize': [True, False]},
                   random_state=42, scoring='accuracy')

In [45]:
random_search.fit(X_train_tfidf, y_train)

print(f"Best hyperparameters: {random_search.best_params_}")

Best hyperparameters: {'binarize': False, 'alpha': np.float64(0.8132160804020101)}


In [47]:
model = BernoulliNB(alpha = np.float64(0.8132160804020101), binarize = False)
model.fit(X_train_tfidf, y_train)
model.fit(X_train_tfidf, y_train)
y_pred = model.predict(X_test_tfidf)
accuracy_score(y_test, y_pred)

0.8603351955307262

El modelo mejora en un 2%.